<a href="https://colab.research.google.com/github/Harshita-2205/-DISTILBERT/blob/main/distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers[torch] evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
import numpy as np
from evaluate import load
from transformers import TrainingArguments, Trainer
from transformers import pipeline

In [3]:
torch.cuda.is_available()

True

In [4]:
# Import the IMDB dataset
imdb = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# Shuffle and select a subset of the training and test data
train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(4000))])
test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])

In [6]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
# Define a function to tokenize the text
def tokenize_function(examples):
   return tokenizer(examples["text"], truncation = True)

# Tokenize the train and test data
tokenized_train = train_dataset.map(tokenize_function, batched = True)
tokenized_test = test_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [8]:
# Define a data collator to handle padding
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [9]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Define a function to compute the evaluation metrics
def compute_metrics(eval_pred):
    # Load accuracy, precision, recall and F1 score metrics
    load_accuracy = load("accuracy") # Use load to load metrics
    load_precision = load("precision")
    load_recall = load("recall")
    load_f1 = load("f1")

    # Unpack logits and labels from eval_pred
    logits, labels = eval_pred

    # Find predictions by taking argmax along the last axis
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy, precision, recall and F1 score using loaded metrics
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    precision = load_precision.compute(predictions=predictions, references=labels)["precision"]
    recall = load_recall.compute(predictions=predictions, references=labels)["recall"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]

    # Return a dictionary containing the computed metrics
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [11]:
# Define the output directory and other training arguments
output_dir_name = "finetuned-sentiment-model-4000-samples-imdb"

training_args = TrainingArguments(
   output_dir = output_dir_name,
   learning_rate = 2e-5,
   per_device_train_batch_size = 16,
   per_device_eval_batch_size = 16,
   num_train_epochs = 2,
   weight_decay = 0.01,
   save_strategy = "epoch",
   push_to_hub = False,
)

# Initialize the trainer
trainer = Trainer(
   model = model,
   args = training_args,
   train_dataset = tokenized_train,
   eval_dataset = tokenized_test,
   tokenizer = tokenizer,
   data_collator = data_collator,
   compute_metrics = compute_metrics,
)

<ipython-input-11-42007c5a2539>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training arguments like learning_rate and batch_size and num_train_epochs were assigned according to Hugging Face recommendations.

In [12]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.282200


TrainOutput(global_step=500, training_loss=0.2821698913574219, metrics={'train_runtime': 454.5951, 'train_samples_per_second': 17.598, 'train_steps_per_second': 1.1, 'total_flos': 1050102185995776.0, 'train_loss': 0.2821698913574219, 'epoch': 2.0})

In [13]:
# Evaluate the model on the test dataset
trainer.evaluate()

{'eval_loss': 0.3005117177963257,
 'eval_accuracy': 0.8833333333333333,
 'eval_precision': 0.8758169934640523,
 'eval_recall': 0.8933333333333333,
 'eval_f1': 0.8844884488448845,
 'eval_runtime': 10.1297,
 'eval_samples_per_second': 29.616,
 'eval_steps_per_second': 1.876,
 'epoch': 2.0}

In [14]:
# Save the model
path = "/content/model"
torch.save(model, path)

In [15]:
# Creating a pipeline for testing the model
model.to('cpu')
class_labels = ['Negative', 'Positive']
model.config.id2label = class_labels
sentiment_model = pipeline(task = 'sentiment-analysis', model = model, tokenizer = tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [16]:
# Testing the model on a sample text
sentiment_model("I love this movie")

[{'label': 'Positive', 'score': 0.9796648025512695}]